In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
import numpy as np

In [2]:
data = pd.read_csv("master_dataset/processed_data.csv")
list(data.columns)
# drop unwanted features

# Dropped all categorical data 

data = data.drop(['title', 'text', 'text_without_stopwords', 'title_without_stopwords','syllables', 'polarity_category', 'overall_content', 'polarity_category_Neutral',
 'polarity_category_Positive'], axis=1)
#'Topic 1 Probability', 'Topic 2 Probability', 'Topic 3 Probbility' , 'Topic 4 Probability' ,'Topic 5 Probability',
#'title_word_count', 'title_sentence_count', 'title_average_word_length','title_punctuation_count', 'title_stopwords_count'  
# 'polarity_category_Neutral' , 'polarity_category_Positive'

In [3]:
#dataset is slightly imbalanced so we will perform upsampling to balance the dataset.
data['class'].value_counts()

0    21196
1    17462
Name: class, dtype: int64

In [4]:
list(data.columns)

['class',
 'text_word_count',
 'title_word_count',
 'text_sentence_count',
 'title_sentence_count',
 'text_average_word_length',
 'title_average_word_length',
 'text_punctuation_count',
 'title_punctuation_count',
 'text_stopwords_count',
 'title_stopwords_count',
 'flesch_readability',
 'subjectivity',
 'polarity',
 'Topic 1 Probability',
 'Topic 2 Probability',
 'Topic 3 Probbility',
 'Topic 4 Probability',
 'Topic 5 Probability']

In [5]:
data.describe()

,class,text_word_count,title_word_count,text_sentence_count,title_sentence_count,text_average_word_length,title_average_word_length,text_punctuation_count,title_punctuation_count,text_stopwords_count,title_stopwords_count,flesch_readability,subjectivity,polarity,Topic 1 Probability,Topic 2 Probability,Topic 3 Probbility,Topic 4 Probability,Topic 5 Probability
count,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000,38658.000000
mean,0.451705,411.374515,12.161209,15.050830,1.067877,5.138347,5.592058,53.119147,1.715635,155.440685,1.136789,0.006674,0.402753,0.056997,0.190953,0.324903,0.125233,0.185360,0.173552
std,0.497669,322.881353,3.765686,11.985707,0.270277,1.875575,1.430398,63.542364,1.452448,123.275458,1.295738,1.003826,0.124806,0.105170,0.281023,0.312565,0.210573,0.258829,0.234861
min,0.000000,1.000000,1.000000,1.000000,1.000000,2.250000,3.000000,0.000000,0.000000,0.000000,0.000000,-83.616811,0.000000,-1.000000,0.000376,0.000246,0.000120,0.000303,0.000268
25%,0.000000,220.000000,10.000000,7.000000,1.000000,4.866522,5.000000,26.000000,1.000000,80.000000,0.000000,-0.169506,0.337127,0.000000,0.002949,0.010969,0.001305,0.003113,0.003024
50%,0.000000,376.000000,11.000000,13.000000,1.000000,5.075727,5.500000,45.000000,2.000000,142.000000,1.000000,0.050549,0.405745,0.054757,0.030471,0.231605,0.006765,0.056382,0.056373
75%,1.000000,522.000000,14.000000,20.000000,1.000000,5.272987,6.066667,67.000000,3.000000,201.000000,2.000000,0.267479,0.475000,0.108144,0.282926,0.584998,0.157612,0.272065,0.268291
max,1.000000,8436.000000,45.000000,321.000000,4.000000,149.000000,149.000000,7295.000000,26.000000,3017.000000,15.000000,2.614284,1.000000,1.000000,0.997819,0.998037,0.996771,0.998911,0.998767


In [8]:
#first split the dataset into training and test sets
x = data.iloc[:,1:]
y = data.iloc[:,:1]

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state = 4222)


x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = 0.25, random_state = 4222)


#balance x_train with oversampling
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy='majority')
# oversample = RandomOverSampler(sampling_strategy = 1)
x_train,y_train = undersample.fit_resample(x_train, y_train)
data = pd.concat([x_train,y_train],axis = 1)

#check that train set is oversampled
data['class'].value_counts()

0    10477
1    10477
Name: class, dtype: int64

In [12]:
x_train

,text_word_count,title_word_count,text_sentence_count,title_sentence_count,text_average_word_length,title_average_word_length,text_punctuation_count,title_punctuation_count,text_stopwords_count,title_stopwords_count,flesch_readability,subjectivity,polarity,Topic 1 Probability,Topic 2 Probability,Topic 3 Probbility,Topic 4 Probability,Topic 5 Probability
0,65,12,3,1,5.587302,4.083333,9,3,23,3,-0.104169,0.350649,0.056818,0.967325,0.008660,0.005838,0.009507,0.008671
1,114,9,3,1,5.106195,5.666667,13,1,40,0,-0.330433,0.442033,-0.105220,0.007751,0.005302,0.975772,0.005816,0.005359
2,725,11,26,1,5.196353,5.090909,85,3,257,4,-0.136771,0.463927,0.041875,0.089378,0.785386,0.000782,0.110923,0.013531
3,401,7,18,1,5.202564,6.285714,44,0,141,1,0.087522,0.299702,0.043446,0.434522,0.340775,0.001140,0.029444,0.194119
4,302,10,11,1,5.156463,5.000000,36,1,119,1,0.018448,0.376786,-0.152976,0.989964,0.002610,0.001779,0.002911,0.002735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20949,565,16,10,1,4.742857,5.533333,46,1,256,0,-0.267928,0.368203,0.122754,0.002057,0.364870,0.000975,0.630648,0.001450
20950,1809,32,144,1,4.418500,4.875000,307,3,772,9,0.852104,0.467778,-0.001175,0.015731,0.304634,0.000364,0.000592,0.678679
20951,418,24,15,2,4.597087,4.500000,37,7,190,7,0.432620,0.487291,0.011708,0.002322,0.914441,0.062097,0.019515,0.001625
20952,677,14,25,1,4.559398,5.538462,86,2,281,0,0.490826,0.482386,0.237027,0.001734,0.258729,0.274193,0.419005,0.046338


In [13]:
# Since we will be performing SVC, and SVC works better on scaled data, we will be scaling all our to ensure that the model runs smoothly
# We will use the Min Max scaler

min_max_scaler = MinMaxScaler()

# Since polarity_category_Neutral and polarity_category_Positive are already from 0 to 1 scale, we will not have to further normalize it.
cols = data.iloc[:,0:-1].columns

# Normalize the data , since the data is not normally distributed we will use minmaxscaler
x_train[cols] = min_max_scaler.fit_transform(x_train[cols]) 

x_test[cols] = min_max_scaler.transform(x_test[cols])
x_validation[cols] = min_max_scaler.transform(x_validation[cols])



In [14]:
x_train.describe()

,text_word_count,title_word_count,text_sentence_count,title_sentence_count,text_average_word_length,title_average_word_length,text_punctuation_count,title_punctuation_count,text_stopwords_count,title_stopwords_count,flesch_readability,subjectivity,polarity,Topic 1 Probability,Topic 2 Probability,Topic 3 Probbility,Topic 4 Probability,Topic 5 Probability
count,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000,20954.000000
mean,0.048848,0.259502,0.043912,0.024339,0.020137,0.016449,0.007345,0.090878,0.051977,0.071321,0.971034,0.407808,0.528809,0.178916,0.340753,0.124398,0.179015,0.178222
std,0.038560,0.088567,0.037323,0.093718,0.012716,0.010084,0.009305,0.076368,0.041459,0.087430,0.011925,0.125182,0.053654,0.274457,0.314603,0.208274,0.253396,0.236795
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.026556,0.204545,0.018750,0.000000,0.018234,0.012346,0.003564,0.052632,0.027179,0.000000,0.968783,0.342050,0.500000,0.002520,0.028118,0.001127,0.002704,0.002850
50%,0.044458,0.250000,0.037500,0.000000,0.019710,0.015873,0.006169,0.105263,0.047398,0.066667,0.971556,0.410740,0.527728,0.023307,0.261930,0.007428,0.052732,0.062568
75%,0.061885,0.318182,0.059375,0.000000,0.021140,0.019400,0.009321,0.157895,0.066954,0.133333,0.974294,0.479956,0.554655,0.250014,0.607903,0.158466,0.259950,0.278387
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Linear SVM

In [15]:
# Linear SVM baseline model

linearSVC = SVC(kernel='linear',random_state=4222)
linearSVC.fit(x_train, np.ravel(y_train))
y_predval_linearSVC = linearSVC.predict(x_validation)

#validation metrics
print("Performance on Validation set:")
print("Accuracy:", metrics.accuracy_score(y_validation, y_predval_linearSVC))
print("Precision:", metrics.precision_score(y_validation, y_predval_linearSVC))
print("Recall:",metrics.recall_score(y_validation, y_predval_linearSVC))
print("F1_score:", metrics.f1_score(y_validation, y_predval_linearSVC))
print("-------------------------------")


#test metrics
y_pred_linearSVC = linearSVC.predict(x_test)
print("Performance on  Test set:")
print("Accuracy:", metrics.accuracy_score(y_test, y_pred_linearSVC))
print("Precision:", metrics.precision_score(y_test, y_pred_linearSVC))
print("Recall:",metrics.recall_score(y_test, y_pred_linearSVC))
print("F1_score:", metrics.f1_score(y_test, y_pred_linearSVC))


Performance on Validation set:
Accuracy: 0.9601655457837558
Precision: 0.9723707664884136
Recall: 0.9383600917431193
F1_score: 0.955062737087832
-------------------------------
Performance on  Test set:
Accuracy: 0.9548629073978272
Precision: 0.9692903995229577
Recall: 0.929653989133543
F1_score: 0.9490585316012261


In [10]:
#hyperparameter tuning with gridsearch for SVM  

grid_params = {
    'C': [0.1,1, 10, 100], 
    'gamma': [1,0.1,0.01,0.001]
    }

scorer = metrics.make_scorer(metrics.f1_score)

gridCV = GridSearchCV(SVC(kernel='linear',random_state = 4222), param_grid = grid_params, cv = 5, scoring = scorer, n_jobs=-1)

gridCV.fit(x_train,np.ravel(y_train))

print("Best Hyper Parameters: ", gridCV.best_params_)

Best Hyper Parameters:  {'C': 100, 'gamma': 1}


In [11]:
finalSVC = SVC( kernel= 'linear', C= 100, gamma= 1, random_state = 4222)
finalSVC.fit(x_train, np.ravel(y_train))
y_predval_finalSVC = finalSVC.predict(x_validation)


#validation metrics
print("Performance on Validation set:")
print("Accuracy:", metrics.accuracy_score(y_validation, y_predval_finalSVC))
print("Precision:", metrics.precision_score(y_validation, y_predval_finalSVC))
print("Recall:",metrics.recall_score(y_validation, y_predval_finalSVC))
print("F1_score:", metrics.f1_score(y_validation, y_predval_finalSVC))
print("-------------------------------")


#test metrics
y_predtest_linearSVC = finalSVC.predict(x_test)
print("Performance on  Test set:")
print("Accuracy:", metrics.accuracy_score(y_test, y_predtest_linearSVC))
print("Precision:", metrics.precision_score(y_test, y_predtest_linearSVC))
print("Recall:",metrics.recall_score(y_test, y_predtest_linearSVC))
print("F1_score:", metrics.f1_score(y_test, y_predtest_linearSVC))


Performance on Validation set:
Accuracy: 0.9636575271598552
Precision: 0.9692712906057945
Recall: 0.9495412844036697
F1_score: 0.9593048515568428
-------------------------------
Performance on  Test set:
Accuracy: 0.9584842214174858
Precision: 0.9681603773584906
Recall: 0.9390906491278238
F1_score: 0.9534039773552039
